In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from google.colab import drive
from array import array
import pandas as pd
import os
import seaborn as sns
drive.mount('/content/gdrive') # 此處需要登入google帳號 
data = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/millan_average13_629_final.csv") #讀取資料集1
data.shape

In [ ]:
data2 = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/taiwan_average37_83_final.csv")#讀取資料集2
data2.shape

In [ ]:
data3 = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/finland_average_47_786_final.csv") #讀取資料集3
data3.shape

In [ ]:
!pip install learn2learn

In [ ]:
!nvidia-smi
!python -V

In [ ]:
import torch
print(torch.__version__)

In [ ]:
df = pd.DataFrame(data)
dataset = df.internet.values.astype(float)

In [ ]:
df2 = pd.DataFrame(data2)
dataset2 = df2.load.values.astype(float)

In [ ]:
df4 = pd.DataFrame(data3)
dataset4 = df4.Internet.values.astype(float)

In [ ]:
dataset1000=np.append(dataset2, dataset4)

In [ ]:
dataset1000=np.append(dataset1000, dataset) #聚合三個資料集

In [ ]:
dataset1000.shape

In [ ]:

dataset1000=dataset1000.reshape(-1,1)  #reshape以利放入模型
dataset1000.shape

In [ ]:
#訓練集測試集產生
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
dataset1000 = sc.fit_transform(dataset1000)

seq_length = 1
x, y = sliding_windows(dataset1000, seq_length)
train_size = 1516090
#test_size = 898761
# train_size = int(len(y) * 0.6)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
#形成元窗格
trainX=trainX.reshape(10,151609,1,1)
trainY=trainY.reshape(10,151609,1,1)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
#GRU宣告
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [ ]:
torch.manual_seed(6)

seed=6

random.seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Remove randomness (may be slower on Tesla GPUs) 
# https://pytorch.org/docs/stable/notes/randomness.html
if seed ==6 :
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
#訓練過程
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import learn2learn as l2l
epochs = 1000
learning_rate = 0.01
meta_learning_rate = 0.01
adaptation_steps = 1
input_dim = 1
hidden_dim = 2
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 1
dropout = 0.2
encoder = GRUModel(input_dim, hidden_dim, layer_dim, output_dim, dropout)
output_layer = nn.Linear(1, 1)
maml = l2l.algorithms.MAML(output_layer, lr=learning_rate, first_order=False)
opt = optim.Adam(list(maml.parameters()) + list(encoder.parameters()), lr=meta_learning_rate)
loss_fn = torch.nn.MSELoss()
for iteration in range(epochs):
    opt.zero_grad()
    iteration_error = 0.0
    for task in range(9):
        learner = maml.clone()
        x_spt = trainX[task]
        y_spt = trainY[task]
        x_qry = trainX[task+1]
        y_qry = trainY[task+1]
        # Fast adapt
        for _ in range(adaptation_steps):
            pred = learner(encoder(x_spt))
            error = loss_fn(pred, y_spt)
            learner.adapt(error)
            print("Epoch: %d, Task: %d, loss: %1.5f" % (iteration, task, error.item()))

        pred = learner(encoder(x_qry))
        evaluation_error = loss_fn(pred, y_qry)
        print(evaluation_error)
        iteration_error += evaluation_error
    # Meta-update the model parameters
    iteration_error /= 10
    print(iteration_error)
    iteration_error.backward()   
    opt.step()


In [ ]:
#結果測試
learner.eval()
test_predict = learner(encoder(testX))
test_predict = test_predict.data.numpy()
testY_plot = testY.data.numpy()
#data_predict=data_predict.reshape(1, -1)
test_predict = sc.inverse_transform(test_predict)
testY_plot = sc.inverse_transform(testY_plot)

#plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(testY_plot)
plt.plot(test_predict)
plt.suptitle('Time-Series Prediction')
plt.show()
import math
testScore = math.sqrt(mean_squared_error(testY_plot, test_predict))
print('Train Score: %.2f RMSE' % (testScore))
#testScore = math.sqrt(mean_squared_error(testY, test_predict))
#print('Test Score: %.2f RMSE' % (testScore))
from math import sqrt

from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

print("mean_absolute_error:",mean_absolute_error(testY_plot, test_predict))

print("mean_squared_error:",mean_squared_error(testY_plot, test_predict))

print("rmse:",sqrt(mean_squared_error(testY_plot, test_predict)))

print("r2 score:",r2_score(testY_plot, test_predict))
print("mape_sk", mean_absolute_percentage_error(testY_plot, test_predict))

In [ ]:
#儲存模型
import joblib
joblib.dump(learner,"/content/gdrive/My Drive/re_thesis/grulearner.pkl")
joblib.dump(encoder,"/content/gdrive/My Drive/re_thesis/gruencoder.pkl")

['/content/gdrive/My Drive/re_thesis/gruencoder.pkl']